In [4]:
import pandas as pd
import pickle as pkl
import numpy as np
import json
import xlsxwriter

In [2]:
with open("models_v1_1216.pkl", "rb") as file1:
    model1 = pkl.load(file1)

with open("models_v2_1216.pkl", "rb") as file2:
    model2 = pkl.load(file2)

---

Json generated with GraphQL that maps the external ID with the name of the sensor. Can be found in the "aux_files" folder.

In [8]:
with open("sens_id_name.json") as file:
    sens_names = json.load(file)
    
sens_id_name = {}
for sens in sens_names['data']['sensors']:
    sens_id_name[sens['externalId']] = sens['name']

---

Recipes data in h5 format. Can be found in the "aux_files" folder.

In [9]:
recipes = pd.read_hdf("recipes_data.h5")

parampredmodels table exported as csv file using pgAdmin:

In [5]:
models_index = pd.read_csv("parampredmodels_1216.csv")
models_index = models_index[['id', 'variant']]

predictions table exported as csv file using pgAdmin:

In [6]:
preds_csv = pd.read_csv("predictions_1216.csv")

In [7]:
tmp = pd.merge(preds_csv, models_index, left_on="param_pred_model_id", right_on="id")
tmp = tmp.iloc[20:, :]

In [8]:
tmp.head()

,id_x,prediction_json,param_pred_model_id,timestamp,comment,current_values_json,id_y,variant
20,21,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,1571829376,NaN,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,ME_EXPL45.100.01
21,22,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,1571829505,NaN,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,ME_EXPL45.100.01
22,23,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.00000000...",7,1571829968,NaN,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,ME_EXPL45.100.01
23,24,"{""704"": 1.0, ""315"": 25.0, ""703"": 2999.99999999...",7,1571830185,NaN,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,ME_EXPL45.100.01
24,25,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,1571830759,NaN,"{""704"": 1.0, ""315"": 25.0, ""703"": 3000.0, ""115""...",7,ME_EXPL45.100.01


In [9]:
preds = pd.DataFrame()

for row in tmp[["prediction_json", "variant", "id_x"]].iterrows():
    aux = pd.DataFrame(json.loads(row[1][0]), index=[0])
    aux["variant"] = row[1][1]
    aux["ID"] = row[1][2]
    preds = pd.concat([preds, aux])

preds.reset_index(inplace=True, drop=True)

In [10]:
preds.head()

,704,315,703,115,114,117,116,111,110,113,...,109,102,103,101,106,107,104,105,variant,ID
0,1.0,25.0,3000.0,65.0,99.067075,65.0,104.067075,65.0,97.286483,65.0,...,50.0,104.231343,65.0,82.441457,50.0,50.0,104.780017,65.0,ME_EXPL45.100.01,21
1,1.0,25.0,3000.0,65.0,99.085487,65.0,104.085487,65.0,97.291153,65.0,...,50.0,104.232407,65.0,82.438539,50.0,50.0,104.779676,65.0,ME_EXPL45.100.01,22
2,1.0,25.0,3000.0,65.0,99.065602,65.0,104.065602,65.0,97.285380,65.0,...,50.0,104.228700,65.0,82.443101,50.0,50.0,104.776750,65.0,ME_EXPL45.100.01,23
3,1.0,25.0,3000.0,65.0,99.068398,65.0,104.068398,65.0,97.283818,65.0,...,50.0,104.223293,65.0,82.444219,50.0,50.0,104.771241,65.0,ME_EXPL45.100.01,24
4,1.0,25.0,3000.0,65.0,97.688861,65.0,102.688861,65.0,97.688861,65.0,...,50.0,105.846897,65.0,82.975589,50.0,50.0,106.316071,65.0,ME_EXPL45.100.01,25


In [23]:
cur_vals = pd.DataFrame()

for row in tmp[["current_values_json", "variant", "id_x"]].iterrows():
    aux = pd.DataFrame(json.loads(row[1][0]), index=[0])
    aux["variant"] = row[1][1]
    aux["ID"] = row[1][2]
    cur_vals = pd.concat([cur_vals, aux], sort=True)

cur_vals.reset_index(inplace=True, drop=True)

In [24]:
cur_vals.head()

,1,101,102,103,104,105,106,107,108,109,...,702,703,704,801,802,803,804,805,ID,variant
0,NaN,82.0,105.0,65.0,105.0,65.0,50.0,50.0,50.0,50.0,...,NaN,3000.0,1.0,NaN,NaN,NaN,NaN,NaN,21,ME_EXPL45.100.01
1,NaN,82.0,105.0,65.0,105.0,65.0,50.0,50.0,50.0,50.0,...,NaN,3000.0,1.0,NaN,NaN,NaN,NaN,NaN,22,ME_EXPL45.100.01
2,NaN,82.0,105.0,65.0,105.0,65.0,50.0,50.0,50.0,50.0,...,NaN,3000.0,1.0,NaN,NaN,NaN,NaN,NaN,23,ME_EXPL45.100.01
3,NaN,82.0,105.0,65.0,105.0,65.0,50.0,50.0,50.0,50.0,...,NaN,3000.0,1.0,NaN,NaN,NaN,NaN,NaN,24,ME_EXPL45.100.01
4,NaN,82.0,105.0,65.0,105.0,65.0,50.0,50.0,50.0,50.0,...,NaN,3000.0,1.0,NaN,NaN,NaN,NaN,NaN,25,ME_EXPL45.100.01


In [26]:
all_variants = cur_vals['variant'].unique().tolist()

In [28]:
# Add here variants that were already checked!
variants_in_v2 = ['ME_EXPL45.100.01', 'ME_EXPL65.085.01', 'ME_EXPL40.100.01', 'ME_EXPL40.100.02', 'ME_EXPL45.085.05', 'ME_EXPL60.075.06', 'ME_EXPL60.100.02']

In [32]:
variants_to_check = list(set(all_variants) - set(variants_in_v2))
variants_to_check

['ME_EXPL45.075.01']

In [34]:
cur_vals_to_check = cur_vals[cur_vals['variant'].isin(variants_to_check)]
cur_vals_to_check

,1,101,102,103,104,105,106,107,108,109,...,702,703,704,801,802,803,804,805,ID,variant
103,NaN,55.0,90.0,65.0,95.0,65.0,90.0,65.0,90.0,65.0,...,NaN,31.0,1.0,NaN,NaN,NaN,NaN,NaN,92,ME_EXPL45.075.01


In [18]:
workbook  = xlsxwriter.Workbook('models_comparison_1216.xlsx')

In [48]:
for row in cur_vals_to_check.iterrows():
    id_ = row[1]["ID"]
    variant = row[1]["variant"]
    print(f"Processing prediction ID {id_}. Variant {variant}.")
    # model1
    scaler1 = model1['scaler']
    scaler1.copy = False
    m1 = model1['models'][variant]['model']
    act_tags = model1['models'][variant]['act_sensor_tags']
    act_tags_str = [str(t) for t in act_tags]
    set_tags = model1['models'][variant]['set_sensor_tags']
    set_tags_str = [str(t) for t in set_tags]
    X = []
    for tag in act_tags_str:
        X.append(row[1][str(tag)])
    if np.any(np.isnan(X)):
        print("Warning NaN found. Impossible to perform a prediction...")
        continue
    scaler1.transform([X])
    X+=[5,5,5]
    X = np.asarray(X)
    X = X.reshape(1,-1)
    print(X)
    p1 = m1.predict(X)
    res1 = dict(zip(set_tags, *p1))

    # model2
    scaler2 = model2['scaler']
    scaler2.copy = False
    m2 = model2['models'][variant]['model']
    act_tags = model2['models'][variant]['act_sensor_tags']
    act_tags_str = [str(t) for t in act_tags]
    set_tags = model2['models'][variant]['set_sensor_tags']
    set_tags_str = [str(t) for t in set_tags]
    X = []
    for tag in act_tags_str:
        X.append(row[1][tag])
    if np.any(np.isnan(X)):
        print("Warning NaN found. Impossible to perform a prediction...")
        continue
    scaler2.transform([X])
    X+=[5,5,5,1,1,1]
    X = np.asarray(X)
    X = X.reshape(1,-1)
    p2 = m2.predict(X)
    res2 = dict(zip(set_tags, *p2))

    #old model
    old_preds = dict(zip(set_tags, *preds[preds['ID']==id_][set_tags_str].values))
    
    #recipe
    aux_recipe = recipes[recipes['maein'] == variant][['ID', 'target_value', 'range_min', 'range_max']]
    
    sens_recipe = aux_recipe.loc[aux_recipe['ID'] == '704',['target_value', 'range_min', 'range_max']].values[0]
    
    worksheet = workbook.add_worksheet(f"{id_}__{variant}")
    worksheet.write(0, 0, "Variant:")
    worksheet.write(0, 1, variant)
    worksheet.write(0, 3, "Prediction ID:")
    worksheet.write(0, 4, id_)
    
    worksheet.write(2, 0, "Sensor ID")
    worksheet.write(2, 1, "Sensor name")
    worksheet.write(2, 2, "RECIPE")
    worksheet.write(2, 3, "OLD PREDICTION")
    worksheet.write(2, 4, "MODEL NEW RATINGS")
    worksheet.write(2, 5, "MODEL MISSING VAL")
    worksheet.write(2, 6, "RANGE MIN")
    worksheet.write(2, 7, "RANGE MAX")
    
    n_line = 3
    for sens_id in set_tags:
        sens_recipe = aux_recipe.loc[aux_recipe['ID'] == str(sens_id),['target_value', 'range_min', 'range_max']].values[0]
        worksheet.write(n_line, 0, sens_id)
        worksheet.write(n_line, 1, sens_id_name[str(sens_id)])
        worksheet.write(n_line, 2, sens_recipe[0])
        worksheet.write(n_line, 3, f"{old_preds[sens_id]:.2f}")
        worksheet.write(n_line, 4, f"{res1[sens_id]:.2f}")
        worksheet.write(n_line, 5, f"{res2[sens_id]:.2f}")
        worksheet.write(n_line, 6, sens_recipe[1])
        worksheet.write(n_line, 7, sens_recipe[2])
        n_line += 1
        
workbook.close()        

Processing prediction ID 92. Variant ME_EXPL45.075.01.
Warning NaN found. Impossible to perform a prediction...


---

# Appendix

## Ratings to h5

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

In [36]:
# TODO: Write the code that dinamically generates the url below using a list of sensors

In [1]:
url = "http://10.41.55.140:8000/sap/opu/odata/sap/ZCAT_IOT_SRV/ZCAT_IFEX_V_MAEIN?$format=json&$filter=spras eq 'DE' and (param eq 'Speed_Main' or param eq 'Temp_Dry0_down' or param eq 'Temp_Dry1_up' or param eq 'Temp_Dry1_down' or param eq 'Temp_Dry2_up' or param eq 'Temp_Dry2_down' or param eq 'Temp_Dry3_up' or param eq 'Temp_Dry3_down' or param eq 'Temp_Dry4_up' or param eq 'Temp_Dry4_down' or param eq 'Temp_Dry5_up' or param eq 'Temp_Dry5_down' or param eq 'Temp_Dry6_up' or param eq 'Temp_Dry6_down' or param eq 'Temp_Dry7_up' or param eq 'Temp_Dry7_down' or param eq 'Temp_Dry8_up' or param eq 'Temp_Dry8_down' or param eq 'Temp_Dry9_up' or param eq 'Temp_Dry9_down' or param eq 'Temp_Dry10_up' or param eq 'Temp_Dry10_down' or param eq 'Taper_Rewind' or param eq 'Speed_Coat0' or param eq 'Speed_Coat1' or param eq 'Speed_Coat2' or param eq 'Speed_Coat3' or param eq 'Speed_Coat4' or param eq 'Speed_Coat5' or param eq 'Speed_Struct5' or param eq 'Power_Corona0' or param eq 'Power_Corona1')"
url

"http://10.41.55.140:8000/sap/opu/odata/sap/ZCAT_IOT_SRV/ZCAT_IFEX_V_MAEIN?$format=json&$filter=spras eq 'DE' and (param eq 'Speed_Main' or param eq 'Temp_Dry0_down' or param eq 'Temp_Dry1_up' or param eq 'Temp_Dry1_down' or param eq 'Temp_Dry2_up' or param eq 'Temp_Dry2_down' or param eq 'Temp_Dry3_up' or param eq 'Temp_Dry3_down' or param eq 'Temp_Dry4_up' or param eq 'Temp_Dry4_down' or param eq 'Temp_Dry5_up' or param eq 'Temp_Dry5_down' or param eq 'Temp_Dry6_up' or param eq 'Temp_Dry6_down' or param eq 'Temp_Dry7_up' or param eq 'Temp_Dry7_down' or param eq 'Temp_Dry8_up' or param eq 'Temp_Dry8_down' or param eq 'Temp_Dry9_up' or param eq 'Temp_Dry9_down' or param eq 'Temp_Dry10_up' or param eq 'Temp_Dry10_down' or param eq 'Taper_Rewind' or param eq 'Speed_Coat0' or param eq 'Speed_Coat1' or param eq 'Speed_Coat2' or param eq 'Speed_Coat3' or param eq 'Speed_Coat4' or param eq 'Speed_Coat5' or param eq 'Speed_Struct5' or param eq 'Power_Corona0' or param eq 'Power_Corona1')"

In [3]:
usr = "USER"
pwd = "SECRET_PASSWORD"

# Commented lines to avoid unwanted connections with wrong credentials
#response = requests.get(url, auth=HTTPBasicAuth(usr, pwd)).text        
#results = json.loads(response)

In [ ]:
df = pd.DataFrame(results['d']['results'])
df = df.drop(['__metadata'], axis=1)
df[['target_value', 'dec_numb', 'dec_count', 'delta_max', 'delta_min', 'delta_alert_max',
       'delta_alert_min']] = df[['target_value', 'dec_numb', 'dec_count', 'delta_max', 'delta_min', 'delta_alert_max',
       'delta_alert_min']].apply(pd.to_numeric)
df.head(5)

In [ ]:
df.columns

In [ ]:
df['ID'] = df.apply(lambda x: sens_id[x['param']], axis=1)
df['name'] = df.apply(lambda x: sens_name[x['param']], axis=1)

In [ ]:
def calc_min_ranges(row):
    min_range = row['delta_min']
    if row['dmin_active']:
        if row['delta_percent']:
            min_range = row['target_value'] * row['delta_min'] / 100
        return row['target_value'] - min_range
    else:
        return "-"

def calc_max_ranges(row):
    max_range = row['delta_max']
    if row['dmax_active']:
        if row['delta_percent']:
            max_range = row['target_value'] * row['delta_max'] / 100
        return row['target_value'] + max_range
    else:
        return "-"

In [ ]:
df.loc[0, 'target_value'] * df.loc[0, 'delta_min'] / 100

In [ ]:
df['range_min'] = df.apply(lambda x: calc_min_ranges(x), axis=1)
df['range_max'] = df.apply(lambda x: calc_max_ranges(x), axis=1)

In [ ]:
df.to_hdf("recipes_data.h5", "df")

---

## GraphQL query

```
{
  sensors {
    externalId
    name
  }
}
```